# AIM
In this notebook we tune a simple CNN to outperform the baseline model. For tuning we use HyperBand (https://arxiv.org/abs/1603.06560). 

## Setting Paths

In [1]:
import sys
from pathlib import Path
#Set root to be the main project folder
root = Path.cwd().parent
print(f"{root=}")

root=PosixPath('/home/gnacikm/Documents/GitHub/Melanoma')


In [2]:
data_path = Path(root/'data')
print(f"{data_path=}")
sav_models = Path(root/'saved_models')
print(f"{sav_models =}")

#Add location of py files to path so we can import
sys.path.insert(0,str(root))

data_path=PosixPath('/home/gnacikm/Documents/GitHub/Melanoma/data')
sav_models =PosixPath('/home/gnacikm/Documents/GitHub/Melanoma/saved_models')


## LIBRARIES / PACKAGES

In [3]:
import os
import json
import tensorflow as tf
import keras_tuner as kt

2021-08-06 09:50:33.697307: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Importing Models

In [4]:
from models.tunning import build_model_seq

### Importing Methods from Utils

In [5]:
from utils.tools import ImageGenerator, FlowFromDir

### Checking if GPU has been detected

In [6]:
device_name = tf.test.gpu_device_name()
if device_name == '':
    raise SystemError("GPU not found")
else:
    print(f"GPU {device_name} found")

GPU /device:GPU:0 found


2021-08-06 09:11:47.936065: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-06 09:11:47.937370: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-06 09:11:47.994607: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 09:11:47.994891: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: GeForce GTX 980 computeCapability: 5.2
coreClock: 1.329GHz coreCount: 16 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 208.91GiB/s
2021-08-06 09:11

## Processing Images

#### Setting paths

In [7]:
dir_with_zip = data_path
dir_with_data = f"{dir_with_zip}/DermMel" 
path_train = f"{dir_with_data}/train_sep/"
path_val = f"{dir_with_data}/valid/"
path_test = f"{dir_with_data}/test/"

#### Using ImageDataGenerator
See https://keras.io/api/preprocessing/image/

In [8]:
data_train = ImageGenerator()
data_val = ImageGenerator()
data_test = ImageGenerator()

In [9]:
train_generator = FlowFromDir(
    data_train,
    path_train,
    shuffle=True)
val_generator = FlowFromDir(
    data_val,
    path_val,
    shuffle=True)
test_generator = FlowFromDir(
    data_test,
    path_test,
    batch_size=1)

Found 10683 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.


### Input and output shapes

In [10]:
input_shape = train_generator[0][0].shape[1:]
input_shape

(160, 160, 3)

In [11]:
num_of_classes = train_generator[0][1].shape[1:][0]
num_of_classes 

2

In [12]:
def factory(input_shape, num_of_classes):
    def model_builder(hp):
        return build_model_seq(hp, input_shape, num_of_classes)
    return model_builder

In [13]:
model_builder = factory(input_shape, num_of_classes)

In [20]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    factor=2,
    max_epochs=5,
    hyperband_iterations=2, 
    project_name=sav_models/'tuner/melanoma_clf'
    )

In [16]:
checkpoint = tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", verbose = 2, mode = 'max', patience=3)
callbacks_list = [checkpoint]

In [17]:
EPOCHS = 15

In [18]:
tuner.search_space_summary()

Search space summary
Default search space size: 13
conv_blocks (Int)
{'default': 3, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
pooling_0 (Choice)
{'default': 'avg', 'conditions': [], 'values': ['avg', 'max'], 'ordered': False}
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
pooling_1 (Choice)
{'default': 'avg', 'conditions': [], 'values': ['avg', 'max'], 'ordered': False}
filters_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
pooling_2 (Choice)
{'default': 'avg', 'conditions': [], 'values': ['avg', 'max'], 'ordered': False}
dropoutConv (Float)
{'default': 0.3, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
n_layers (Int)
{'default': 2, 'conditions': [], 'min_value': 1, 

In [17]:
tuner.search(train_generator,
             steps_per_epoch=train_generator.n // train_generator.batch_size,
             epochs=EPOCHS,
             validation_data=val_generator,
             validation_steps=val_generator.n // val_generator.batch_size,
             callbacks=[callbacks_list])

Trial 42 Complete [00h 08m 37s]
val_accuracy: 0.8372747898101807

Best val_accuracy So Far: 0.8606418967247009
Total elapsed time: 03h 09m 55s
INFO:tensorflow:Oracle triggered exit


In [18]:
model = tuner.get_best_models(1)[0]

In [19]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
best_hyperparameters

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 158, 158, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 158, 158, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 77, 77, 128)       36992     
_________________________________________________________________
batch_normalization_1 (Batch (None, 77, 77, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 64)        7

In [21]:
model.evaluate(test_generator, steps = test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 32s 9ms/step - loss: 0.3452 - accuracy: 0.8531


[0.3452340364456177, 0.853131115436554]

In [22]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    sav_models/"weights/3ConvTuned.h5", 
    monitor = "val_accuracy", 
    verbose = 2, 
    save_weights_only=True,
    save_best_only = True,
    mode = 'max'
)

In [29]:
#After epoch 7 it starts overfitting
history = model.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    epochs=EPOCHS, 
                    validation_data=val_generator, 
                    validation_steps=val_generator.n//val_generator.batch_size,
                    callbacks=[checkpoint])

In [9]:
model.load_weights(sav_models/"weights/3ConvTuned.h5")

In [27]:
model.evaluate(test_generator, steps = test_generator.n//test_generator.batch_size)

3561/3561 [==============================] - 33s 9ms/step - loss: 0.2053 - accuracy: 0.9174


[0.20532256364822388, 0.9174389243125916]

In [10]:
model_json = model.to_json()
with open(sav_models/"json/3ConvTuned.json", "w") as json_file:
    json_file.write(model_json)